## Домашняя работа №1
ФИО: Федоров Александр Дмитриевич

## Задание 1
Реализуйте метакласс ThreadSafeSingleton, который обеспечивает создание только одного экземпляра класса, даже в многопоточной среде. Используйте from threading import Lock

In [1]:
from threading import Lock
class ThreadSafeSingleton(type):
    _instances = {}
    lock = Lock()
    
    def __call__(cls, *args, **kwargs):
        cls.lock.acquire()
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        cls.lock.release()
        return cls._instances[cls]

In [2]:
class DatabasePool(int, metaclass=ThreadSafeSingleton):
    def __init__(self):
        self.connection = 'connected'

    def get_connection(self):
        return self.connection

Проверка задания 1

In [3]:
# Создайте 3 экземпляра DatabasePool
pool1 = DatabasePool()
pool2 = DatabasePool()
pool3 = DatabasePool()

# Убедитесь, что это один и тот же объект
assert pool1 is pool2 is pool3

In [4]:
# Проверьте, что соединения разделяются между экземплярами
conn1 = pool1.get_connection()
pool2.connection = 'disconnected'
conn2 = pool2.get_connection()

assert conn1 != conn2

## Задание 2
Создайте метакласс, который считает, сколько раз создавался каждый класс.

Требования:

Метакласс должен иметь атрибут _counters
При создании экземпляра класса счетчик должен увеличиваться
Добавьте метод get_count(), который возвращает количество созданных экземпляров

In [5]:
class ObjectCounter(type):
    _counters = 0
    
    def __call__(cls, *args, **kwargs):
        cls._counters += 1

    def get_count(cls):
        return cls._counters

In [6]:
class User(int, metaclass=ObjectCounter):
    def __init__(self, name):
        self.name = name

class Product(int, metaclass=ObjectCounter):
    def __init__(self, name):
        self.name = name

Проверка задание 2

In [7]:
# Проверка
user1 = User("Alice")
user2 = User("Bob")
product1 = Product("Laptop")

print(User.get_count())
print(Product.get_count())

2
1


## Задание 3
Создайте метакласс, который автоматически добавляет метод describe() в каждый класс.

Требования:

Метод describe() должен возвращать строку с именем класса
Используйте метакласс для создания классов Car и Book

In [8]:
def describe(self):
    return f'Это объект класса {self.__class__.__name__}'

class ClassName(type):
    
    def __new__(cls, name, bases, attrs):
        attrs.update({"describe": describe})
        return super().__new__(cls, name, bases, attrs)    

In [9]:
class Car(metaclass=ClassName):
    def __init__(self, name):
        self.name = name

class Book(metaclass=ClassName):
    def __init__(self, name):
        self.name = name

Проверка задание 3

In [10]:
car = Car("Toyota")
book = Book("Python для начинающих")

print(car.describe())  # Должно быть "Это объект класса Car"
print(book.describe()) # Должно быть "Это объект класса Book"

Это объект класса Car
Это объект класса Book


## Задание 4
Создайте метакласс, который проверяет, что у класса есть метод save(). Можно использовать __new__

Требования:

Если у класса нет метода save(), метакласс должен выдать ошибку
Создайте класс User с методом save()
Попробуйте создать класс Message без метода save() (должна быть ошибка)

In [11]:
class MyException(Exception):
    def __init__(self, message=f'Новый класс не содержит метода save'):
        self.message = message
        super().__init__(self.message)

class SaveMeta(type):
    def __new__(cls, name, bases, attrs):
        if 'save' not in attrs:
            raise MyException
            
        return super().__new__(cls, name, bases, attrs)    

Проверка задания 4

In [12]:
class User(metaclass=SaveMeta):
     def __init__(self, name):
         self.name = name

     def save(self):
         return f'Есть метод save'

In [13]:
user = User("Alice")
user.save()  # Должно работать

'Есть метод save'

In [14]:
# Этот код должен вызвать ошибку:
class Message(metaclass=SaveMeta):
     def __init__(self, text):
         self.text = text

MyException: Новый класс не содержит метода save